In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import tensorflow as tf

from keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from sklearn.model_selection import StratifiedKFold
import pandas as pd 

from scipy import stats
from tensorflow.keras.models import save_model

2023-06-23 15:40:48.277583: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# 원본 데이터 load

Dataset_origin = np.load('images_origin.npy')
Labels_origin = np.load('labels_origin.npy')

Dataset_origin.shape, Labels_origin.shape

((3000, 224, 224, 3), (3000,))

In [3]:
# 원본 데이터 + 좌우 반전된 데이터 load

Dataset_flipped = np.load('images_flipped.npy')
Labels_flipped = np.load('labels_flipped.npy')

Dataset_flipped.shape, Labels_flipped.shape

((6000, 224, 224, 3), (6000,))

In [4]:
# 원본 데이터 + 좌우 반전된 데이터 + 밝기 조절 데이터 load

Dataset_brightness = np.load('images_brightness.npy')
Labels_brightness = np.load('labels_brightness.npy')

Dataset_brightness.shape, Labels_brightness.shape

((9000, 224, 224, 3), (9000,))

In [5]:
# train / val / Test data 분리 - Dataset 1 

origin_train, origin_test, labels_train, labels_test = train_test_split(Dataset_origin, Labels_origin, 
                                                            test_size=0.2,stratify=Labels_origin, random_state=42)

origin_train, origin_val, labels_train, labels_val = train_test_split(origin_train, labels_train, 
                                                            test_size=0.2,stratify=labels_train, random_state=42)

print(origin_train.shape, origin_val.shape, origin_test.shape)
print(labels_train.shape, labels_val.shape, labels_test.shape)

(1920, 224, 224, 3) (480, 224, 224, 3) (600, 224, 224, 3)
(1920,) (480,) (600,)


In [6]:
# train / val / Test data 분리 - Dataset 2

flipped_train, flipped_test, labels_flipped_train, labels_flipped_test = train_test_split(Dataset_flipped, Labels_flipped, 
                                                                                    test_size=0.2,stratify=Labels_flipped, random_state=42)

flipped_train, flipped_val, labels_flipped_train, labels_flipped_val = train_test_split(flipped_train, labels_flipped_train, 
                                                                                    test_size=0.2,stratify=labels_flipped_train, random_state=42)

print(flipped_train.shape, flipped_val.shape, flipped_test.shape)
print(labels_flipped_train.shape, labels_flipped_val.shape, labels_flipped_test.shape)

(3840, 224, 224, 3) (960, 224, 224, 3) (1200, 224, 224, 3)
(3840,) (960,) (1200,)


In [7]:
# train / val / Test data 분리 - Dataset 3

bright_train, bright_test, labels_bright_train, labels_bright_test = train_test_split(Dataset_brightness, Labels_brightness, 
                                                                                    test_size=0.2,stratify=Labels_brightness, random_state=42)

bright_train, bright_val, labels_bright_train, labels_bright_val = train_test_split(bright_train, labels_bright_train, 
                                                                                    test_size=0.2,stratify=labels_bright_train, random_state=42)

print(bright_train.shape, bright_val.shape, bright_test.shape)
print(labels_bright_train.shape, labels_bright_val.shape, labels_bright_test.shape)

(5760, 224, 224, 3) (1440, 224, 224, 3) (1800, 224, 224, 3)
(5760,) (1440,) (1800,)


In [10]:
CNN_Model_Dataset1  = tf.keras.Sequential([
                            tf.keras.layers.Conv2D(2, (1, 1), activation='relu', input_shape=(224, 224, 3)),
                            tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
                            tf.keras.layers.MaxPooling2D((2, 2)),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(128, activation='relu'),
                            tf.keras.layers.Dropout(0.5),
                            tf.keras.layers.Dense(64, activation='relu'),
                            tf.keras.layers.Dropout(0.5),
                            tf.keras.layers.Dense(1, activation='sigmoid')
                            ])

CNN_Model_Dataset1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
CNN_Model_Dataset1.fit(origin_train, labels_train, epochs=30, validation_data=(origin_val, labels_val), callbacks=[early_stopping])

Epoch 1/30
60/60 [==============================] - 26s 409ms/step - loss: 0.8148 - accuracy: 0.6615 - val_loss: 0.3333 - val_accuracy: 0.9146
Epoch 2/30
60/60 [==============================] - 24s 409ms/step - loss: 0.2389 - accuracy: 0.9068 - val_loss: 0.0693 - val_accuracy: 0.9771
Epoch 3/30
60/60 [==============================] - 24s 404ms/step - loss: 0.1126 - accuracy: 0.9667 - val_loss: 0.0612 - val_accuracy: 0.9833
Epoch 4/30
60/60 [==============================] - 23s 376ms/step - loss: 0.0776 - accuracy: 0.9734 - val_loss: 0.0467 - val_accuracy: 0.9854
Epoch 5/30
60/60 [==============================] - 23s 391ms/step - loss: 0.0707 - accuracy: 0.9818 - val_loss: 0.0461 - val_accuracy: 0.9854
Epoch 6/30
60/60 [==============================] - 25s 412ms/step - loss: 0.0872 - accuracy: 0.9703 - val_loss: 0.0370 - val_accuracy: 0.9896
Epoch 7/30
60/60 [==============================] - 21s 356ms/step - loss: 0.0547 - accuracy: 0.9844 - val_loss: 0.0388 - val_accuracy: 0.9896

In [11]:
CNN_Model_Dataset2  = tf.keras.Sequential([
                            tf.keras.layers.Conv2D(2, (1, 1), activation='relu', input_shape=(224, 224, 3)),
                            tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
                            tf.keras.layers.MaxPooling2D((2, 2)),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(128, activation='relu'),
                            tf.keras.layers.Dropout(0.5),
                            tf.keras.layers.Dense(64, activation='relu'),
                            tf.keras.layers.Dropout(0.5),
                            tf.keras.layers.Dense(1, activation='sigmoid')
                            ])

CNN_Model_Dataset2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
CNN_Model_Dataset2.fit(flipped_train, labels_flipped_train, epochs=30, validation_data=(flipped_val, labels_flipped_val), callbacks=[early_stopping])

Epoch 1/30
120/120 [==============================] - 54s 438ms/step - loss: 0.3011 - accuracy: 0.8885 - val_loss: 0.0701 - val_accuracy: 0.9740
Epoch 2/30
120/120 [==============================] - 48s 399ms/step - loss: 0.0828 - accuracy: 0.9763 - val_loss: 0.0484 - val_accuracy: 0.9760
Epoch 3/30
120/120 [==============================] - 55s 460ms/step - loss: 0.0630 - accuracy: 0.9849 - val_loss: 0.0322 - val_accuracy: 0.9885
Epoch 4/30
120/120 [==============================] - 50s 419ms/step - loss: 0.0576 - accuracy: 0.9859 - val_loss: 0.0825 - val_accuracy: 0.9719
Epoch 5/30
120/120 [==============================] - 49s 408ms/step - loss: 0.0413 - accuracy: 0.9904 - val_loss: 0.0262 - val_accuracy: 0.9927
Epoch 6/30
120/120 [==============================] - 49s 411ms/step - loss: 0.0433 - accuracy: 0.9919 - val_loss: 0.0295 - val_accuracy: 0.9937
Epoch 7/30
120/120 [==============================] - 56s 464ms/step - loss: 0.0405 - accuracy: 0.9930 - val_loss: 0.0217 - val_ac

In [12]:
CNN_Model_Dataset3  = tf.keras.Sequential([
                            tf.keras.layers.Conv2D(2, (1, 1), activation='relu', input_shape=(224, 224, 3)),
                            tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
                            tf.keras.layers.MaxPooling2D((2, 2)),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(128, activation='relu'),
                            tf.keras.layers.Dropout(0.5),
                            tf.keras.layers.Dense(64, activation='relu'),
                            tf.keras.layers.Dropout(0.5),
                            tf.keras.layers.Dense(1, activation='sigmoid')
                            ])

CNN_Model_Dataset3.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
CNN_Model_Dataset3.fit(bright_train, labels_bright_train, epochs=30, validation_data=(bright_val, labels_bright_val), callbacks=[early_stopping])

Epoch 1/30
180/180 [==============================] - 97s 526ms/step - loss: 0.8693 - accuracy: 0.5844 - val_loss: 0.4349 - val_accuracy: 0.6382
Epoch 2/30
180/180 [==============================] - 83s 458ms/step - loss: 0.3422 - accuracy: 0.8378 - val_loss: 0.1039 - val_accuracy: 0.9618
Epoch 3/30
180/180 [==============================] - 73s 407ms/step - loss: 0.2035 - accuracy: 0.9148 - val_loss: 0.0940 - val_accuracy: 0.9764
Epoch 4/30
180/180 [==============================] - 77s 430ms/step - loss: 0.1325 - accuracy: 0.9363 - val_loss: 0.0841 - val_accuracy: 0.9708
Epoch 5/30
180/180 [==============================] - 79s 436ms/step - loss: 0.1453 - accuracy: 0.9307 - val_loss: 0.1225 - val_accuracy: 0.9639
Epoch 6/30
180/180 [==============================] - 82s 458ms/step - loss: 0.0988 - accuracy: 0.9606 - val_loss: 0.0989 - val_accuracy: 0.9722
Epoch 7/30
180/180 [==============================] - 79s 440ms/step - loss: 0.0953 - accuracy: 0.9628 - val_loss: 0.0769 - val_ac

In [13]:
TestData = np.load('images_test.npy')
TestLabel = np.load('labels_test.npy')

In [14]:
_, score_Dataset1 = CNN_Model_Dataset1.evaluate(TestData, TestLabel)
_, score_Dataset2 = CNN_Model_Dataset2.evaluate(TestData, TestLabel)
_, score_Dataset3 = CNN_Model_Dataset3.evaluate(TestData, TestLabel)

score_Dataset1, score_Dataset2, score_Dataset3

25/25 [==============================] - 3s 114ms/step - loss: 0.1333 - accuracy: 0.9663


(0.9775000214576721, 0.9862499833106995, 0.9662500023841858)

In [15]:
# 각 데이터셋을 학습한 모델 저장

CNN_Model_Dataset1.save("CNN_Model_Dataset1.h5")
CNN_Model_Dataset2.save("CNN_Model_Dataset2.h5")
CNN_Model_Dataset3.save("CNN_Model_Dataset3.h5")